# Web Scraping Code for Anime Images

In [1]:
import requests
import bs4
import pandas as pd
import numpy as np
import time

## Part 1: Fetch URL of Pic to be Scraped

In [2]:
#  Read in Data Scraped from myanimelist.net
df = pd.read_csv('data/dataset_EDA_Topic_14022021.csv')

C:\Users\Sam Too\anaconda3\envs\deep\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12,13,22,50,51,52,53,54,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,80,81,82,83,84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(df.shape)
df.head()

(17489, 91)


,index,Unnamed: 0,Unnamed: 0.1,Title,URL,English,Synonyms,Japanese,Type,Episodes,...,Studio_3,Studio_4,Studio_5,Studio_6,Anime_Number,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Topic_Category
0,0,0,0,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",鋼の錬金術師 FULLMETAL ALCHEMIST,TV,64.0,...,NaN,NaN,NaN,NaN,0,0.0,0.2993,"world, human, power, earth, fight, save, war, ...",in order for something to be obtained somethi...,Anime about war or battles on earth
1,1,1,1,Shingeki no Kyojin: The Final Season,https://myanimelist.net/anime/40028/Shingeki_n...,Attack on Titan Final Season,"Shingeki no Kyojin Season 4, Attack on Titan S...",進撃の巨人 The Final Season,TV,16.0,...,NaN,NaN,NaN,NaN,1,0.0,0.2820,"world, human, power, earth, fight, save, war, ...",gabi braun and falco grice have been training ...,Anime about war or battles on earth
2,2,2,2,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,Steins;Gate,NaN,STEINS;GATE,TV,24.0,...,NaN,NaN,NaN,NaN,2,1.0,0.3302,"year, group, team, member, lead, club, face, j...",the self proclaimed mad scientist rintarou oka...,Anime about membership in a club or group
3,3,3,3,Shingeki no Kyojin Season 3 Part 2,https://myanimelist.net/anime/38524/Shingeki_n...,Attack on Titan Season 3 Part 2,NaN,進撃の巨人 Season3 Part.2,TV,10.0,...,NaN,NaN,NaN,NaN,3,1.0,0.3661,"year, group, team, member, lead, club, face, j...",seeking to restore humanity s diminishing hope...,Anime about membership in a club or group
4,4,4,4,Gintama°,https://myanimelist.net/anime/28977/Gintama°,Gintama Season 4,Gintama' (2015),銀魂°,TV,51.0,...,NaN,NaN,NaN,NaN,4,1.0,0.2368,"year, group, team, member, lead, club, face, j...",gintoki shinpachi and kagura return as the fun...,Anime about membership in a club or group


In [4]:
# add /pics to form the url in which the pic will be scraped from
df['url_img'] = df['URL'].astype(str) + '/pics'

## Part 2: Codes to Scrape Pic

In [5]:
img_url_full = df['url_img'].tolist()
index_img = df['index'].tolist()
img_url_with_index = [(index_img[i], img_url_full[i]) for i in range(len(img_url_full))]

In [6]:
for i in range(0,10):
    print(img_url_with_index[i])

(0, 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/pics')
(1, 'https://myanimelist.net/anime/40028/Shingeki_no_Kyojin__The_Final_Season/pics')
(2, 'https://myanimelist.net/anime/9253/Steins_Gate/pics')
(3, 'https://myanimelist.net/anime/38524/Shingeki_no_Kyojin_Season_3_Part_2/pics')
(4, 'https://myanimelist.net/anime/28977/Gintama°/pics')
(5, 'https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011/pics')
(6, 'https://myanimelist.net/anime/9969/Gintama/pics')
(7, 'https://myanimelist.net/anime/820/Ginga_Eiyuu_Densetsu/pics')
(8, 'https://myanimelist.net/anime/15417/Gintama__Enchousen/pics')
(9, 'https://myanimelist.net/anime/35180/3-gatsu_no_Lion_2nd_Season/pics')


In [7]:
import os
# create new directory
if not os.path.exists('data/images'):
    os.makedirs('data/images')

image_dir = r'data/images/'

In [8]:
# initiliase counter to tag image scraped to corresponding 
# why is there a need for counter?
download_done = []
download_fail = []
counter = 0

for i in img_url_with_index:
    
    link = i[1]
    counter += 1
    
    # sleep for 180 every 100 pictures
    if counter % 100 == 0:
        time.sleep(180)
    
    try:
        url = requests.get(link) # main site 
        soup = bs4.BeautifulSoup(url.text,'lxml') 
        img_url = soup.select('.lazyload')[0]['data-src'] # fetch img url 
        img = requests.get(img_url) # fetch img
        
        filename = image_dir + 'index_' + str(i[0]) + '.jpg'
        with open(filename, 'wb') as f:
            f.write(img.content)
        print('index_' + str(i[0]) + ' image downloaded.')
        download_done.append(i)
        
    except:
        print('index_' + str(i[0]) + ' image not downloaded!')
        download_fail.append(i)
        continue

index_0 image downloaded.
index_1 image downloaded.
index_2 image downloaded.
index_3 image downloaded.
index_4 image downloaded.
index_5 image downloaded.
index_6 image downloaded.
index_7 image downloaded.
index_8 image downloaded.
index_9 image downloaded.
index_10 image downloaded.
index_11 image downloaded.
index_12 image downloaded.
index_13 image downloaded.
index_14 image downloaded.
index_15 image downloaded.
index_16 image downloaded.
index_17 image downloaded.
index_18 image downloaded.
index_19 image downloaded.
index_20 image downloaded.
index_21 image downloaded.
index_22 image downloaded.
index_23 image downloaded.
index_24 image downloaded.
index_25 image downloaded.
index_26 image downloaded.
index_27 image downloaded.
index_28 image downloaded.
index_29 image downloaded.
index_30 image downloaded.
index_31 image downloaded.
index_32 image downloaded.
index_33 image downloaded.
index_34 image downloaded.
index_35 image downloaded.
index_36 image downloaded.
index_37 im

In [9]:
#save files
import pickle
with open('download_done.pickle', 'wb') as f:
    pickle.dump(download_done, f)

with open('download_fail.pickle', 'wb') as f:
    pickle.dump(download_fail, f)


## Part 3. Fetch Index of Images which failed to download and download them

In [ ]:
import os
import re
import numpy as np

In [ ]:
# import download_fail
with open('download_fail.pickle', 'rb') as f:
    download_fail = pickle.load(f)

In [10]:
download_fail

[(997,
  'https://myanimelist.net/anime/10092/Break_Blade_6__Doukoku_no_Toride/pics'),
 (998, 'https://myanimelist.net/anime/3841/Chis_Sweet_Home/pics'),
 (1793,
  'https://myanimelist.net/anime/28405/Ansatsu_Kyoushitsu__Deai_no_Jikan/pics'),
 (1794, 'https://myanimelist.net/anime/59/Chobits/pics'),
 (1797, 'https://myanimelist.net/anime/36561/Doupo_Cangqiong_Specials/pics'),
 (1798, 'https://myanimelist.net/anime/34933/Kakegurui/pics'),
 (3795,
  'https://myanimelist.net/anime/1798/Animation_Seisaku_Shinkou_Kuromi-chan_2/pics'),
 (3796, 'https://myanimelist.net/anime/5342/Asura_Cryin/pics'),
 (3798,
  'https://myanimelist.net/anime/2206/Galaxy_Angel_Music_Collection__Shouen_to_Shien_no_Cassoulet/pics'),
 (4298, 'https://myanimelist.net/anime/280/Animal_Yokochou/pics'),
 (7090,
  'https://myanimelist.net/anime/42044/Minegishi-san_wa_Ootsu-kun_ni_Tabesasetai/pics'),
 (7092, 'https://myanimelist.net/anime/20539/Nameko-ke_no_Ichizoku/pics'),
 (7093, 'https://myanimelist.net/anime/34003/Na

In [11]:
download_done_again = []
download_fail_again = []
counter = 0

for i in download_fail:
    
    link = i[1]
    counter += 1
    
    # sleep for 180 every 100 pictures
    if counter % 100 == 0:
        time.sleep(180)
    
    try:
        url = requests.get(link) # main site 
        soup = bs4.BeautifulSoup(url.text,'lxml') 
        img_url = soup.select('.lazyload')[0]['data-src'] # fetch img url 
        img = requests.get(img_url) # fetch img
        
        filename = image_dir + 'index_' + str(i[0]) + '.jpg'
        with open(filename, 'wb') as f:
            f.write(img.content)
        print('index_' + str(i[0]) + ' image downloaded.')
        download_done_again.append(i)
        
    except:
        print('index_' + str(i[0]) + ' image not downloaded!')
        download_fail_again.append(i)
        continue

index_997 image downloaded.
index_998 image downloaded.
index_1793 image downloaded.
index_1794 image downloaded.
index_1797 image downloaded.
index_1798 image downloaded.
index_3795 image downloaded.
index_3796 image downloaded.
index_3798 image downloaded.
index_4298 image downloaded.
index_7090 image downloaded.
index_7092 image downloaded.
index_7093 image downloaded.
index_7094 image downloaded.
index_7097 image downloaded.
index_9897 image not downloaded!
index_10095 image downloaded.
index_10097 image downloaded.
index_10791 image downloaded.
index_10793 image downloaded.
index_10794 image downloaded.
index_10795 image downloaded.
index_10797 image downloaded.
index_10798 image downloaded.
index_10992 image downloaded.
index_10993 image downloaded.
index_10994 image downloaded.
index_10997 image downloaded.
index_11047 image not downloaded!
index_11053 image not downloaded!
index_11092 image not downloaded!
index_11100 image not downloaded!
index_11147 image not downloaded!
inde

In [12]:
download_fail_again

[(9897,
  'https://myanimelist.net/anime/17387/Imokawa_Mukuzo__Genkanban_no_Maki/pics'),
 (11047, 'https://myanimelist.net/anime/33826/Neko_no_Kuni_no_Kenpou/pics'),
 (11053, 'https://myanimelist.net/anime/24573/Neko_to_Nezumi/pics'),
 (11092, 'https://myanimelist.net/anime/36324/Nijiiro_no_Fushigina_Ishi/pics'),
 (11100, 'https://myanimelist.net/anime/38879/Ningen_Moyou/pics'),
 (11147,
  'https://myanimelist.net/anime/7495/Nonki_na_Tou-san_Ryuuguu_Mairi/pics'),
 (11299, 'https://myanimelist.net/anime/35024/Onita_no_Boushi/pics'),
 (11316, 'https://myanimelist.net/anime/43598/Ookina_Furudokei/pics'),
 (11367,
  'https://myanimelist.net/anime/23181/Otogi_Banashi__Bunbuku_Chagama/pics'),
 (11392,
  'https://myanimelist.net/anime/39254/Oyako_de_Manabu__Kaiji_no_Youjin_-_Jishin_no_Youjin/pics'),
 (11469,
  'https://myanimelist.net/anime/39265/Peter-kun_no_Hinotama_Taiji/pics'),
 (11470,
  'https://myanimelist.net/anime/39624/Peter-kun_no_Koutsuu_Anzen/pics'),
 (11532, 'https://myanimelist

In [13]:
#save files
import pickle
with open('download_done_again.pickle', 'wb') as f:
    pickle.dump(download_done_again, f)

with open('download_fail_again.pickle', 'wb') as f:
    pickle.dump(download_fail_again, f)